In [73]:
import time
import requests 
import hmac
import json
import hashlib
import numpy as np
from datetime import datetime
import pandas as pd
import gspread
import os
from datetime import datetime
#----------
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
#----------
from IPython.display import clear_output

# API info
API_HOST = 'https://api.bitkub.com'
API_KEY = config['API']['key']
API_SECRET = bytes(config['API']['secret'], 'utf-8') 

symbol = config['GRID_SYSTEM']['symbol']

In [ ]:
#-----------------------------------Bitkub---------------------------------------------#

In [74]:
class bitkubAPI():
    def __init__(self,host,api_key,api_secret):
        self.API_HOST = host
        self.API_KEY = api_key
        self.API_SECRET = api_secret
        self.header = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'X-BTK-APIKEY': self.API_KEY,
            }
        
    def timestamp(self,tm):
        date = datetime.datetime.strptime(tm, "%Y-%m-%d")
        ts = time.mktime(date.timetuple())
        return ts
        
    def _json_encode(self,data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def _sign(self,data):
        j = self._json_encode(data)
        h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()
    
    def _get(self,url,**kwargs): 
        try:
            res = requests.get(self.API_HOST+ url,**kwargs)
            res = json.loads(res.text)
            res = res
            return res
        except:
            return res
    
    def _post(self,url,data): 
        try:
            signature = self._sign(data)
            data['sig'] = signature
            res = requests.post(self.API_HOST + url, headers= self.header, data=self._json_encode(data))
            return res
        except:
            return res
        
    def api_status(self):  
        return self._get('/api/status')
        
    def serverTime(self):
        res = requests.get(self.API_HOST + '/api/servertime')
        return json.loads(res.text)
    
    def bids(self,sym):
        return self._get('/api/market/bids?sym='+sym+'&lmt=1')

    def asks(self,sym):
        return self._get('/api/market/asks?sym='+sym+'&lmt=1')
    
    def balance(self):
        data = {
            'ts': self.getServerTime(),
        }
        return self._post('/api/market/balances',data)
    
    def opened_orders(self,sym):
        data = {
            'sym': sym,
        }
        return self._post('/api/market/balances',data)
    
    def info_orders(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/balances',data)
    
    def cancel_order(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/cancel-order',data)
    
    def his_price(self,sym,frm):
        params = 'sym='+sym
        params = params + '&int=86400'
        params = params + f'&frm={int(self.timestamp(frm))}'
        params = '/api/market/tradingview?'+params
        return self._get(params)
    
    def place_order(self,sym,side,amt,rat,typ):
        data = {
            'sym': sym,
            'amt':amt,
            'rat':rat,
            'typ':typ,
        }
        if(side =='buy'):
            return self._post('/api/market/place-ask/test',data)
        elif(side =='sell'):
            return self._post('/api/market/place-bid/test',data)

In [ ]:
#-----------------------------------FTX---------------------------------------------#

In [ ]:
import time
import hmac
from typing import Optional, Dict, Any, List
from requests import Request, Session, Response

class ftxAPI():
    def __init__(self):
        self._api_host = 'https://ftx.com/api'
        self._api_key = b'1vyg6pC1WY-ODMhSyeHasYBV8ZVrgpAKYEu4uqWZ'
        self._api_secret  = 'qXuNtAcsQfZZEbzHlrm_rdBOtCrGgFHrYjdkshw8'
        self.ts = int(time.time() * 1000)
        self._subaccount_name = ""
        self._session = Session()
        
    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)
    
    def _post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('POST', path, params=params)
    
    def _request(self, method: str, path: str, **kwargs) -> Any:
        request = Request(method, self._api_host + path, **kwargs)
        self._sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _sign_request(self, request: Request) -> None:
        self.ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{self.ts}{prepared.method}{prepared.path_url}'.encode()
        if prepared.body:
            signature_payload += prepared.body
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(self.ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)
    
    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    
    def get_ticker(self,market_name):
        return self._get(f'/markets/{market_name}/orderbook?depth={1}')

    def place_orders(self,market,side,size,price,type_ord):
        return self._post(f'/orders',{   'market': market,
                                         'side': side,
                                         'price': price,
                                         'size': size,
                                         'type': type_ord,
                                     })
    
    def place_conditional_orders(self,market,side,size,type_ord,tp):
        return self._post(f'/conditional_orders',{    'market': market, 
                                                      'side': side,
                                                      'size': size,
                                                      'type': type_ord,
                                                      'triggerPrice':tp
                                                      'orderPrice':tp
                                                 })
    
    def historicalPrice(self,market_name,resolution,limit,start_time,end_time):
        return self._get(f'/markets/{market_name}/candles?resolution={resolution}&limit={limit}&start_time={start_time}&end_time={end_time}')
    
    def get_open_orders(self,market):
        return self._get(f'/orders?market={market}')
    
    def get_open_conditional_order(self,market):
        return self._get(f'/conditional_orders?market={market}')
    
    def get_conditional_order(self,conditional_order_id):
        return self._get(f'/conditional_orders/{conditional_order_id}/triggers')

In [75]:
#--------------------------------------------------------------------------------#

In [ ]:
class symbol():
    def __init__(self,sym,API):
        self.ticker = {'ask':'','bid':'','askv':'','bidv':''}
        self.order = {}

        gc = gspread.service_account(os.getcwd()+'\\service_account.json')
        self.sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/16sEZIsXLYYOrjVKo8xLOpusOeeYGAmnw_Xnrm0y2qwg/edit?usp=drive_web&ouid=112544876186489015703')

        
    def get_ticker(symbol):
        try:
            self.ticker['bid'] = float(API.ticker(sym)['bids'][0][0])
            self.ticker['bidv'] = float(API.ticker(sym)['bids'][0][1])
            self.ticker['ask'] = float(API.ticker(sym)['asks'][0][0])
            self.ticker['askv'] = float(API.ticker(sym)['asks'][0][1])
            return True
        except:
            return False
            
    def load_order(self):
        worksheet = self.sheet.worksheets()
        #check sheet
        if (self.symbol not in str(worksheet)):
            #create sheert
            self.sheet.add_worksheet(title=f"{sym}", rows="100", cols="13")
            self.sheet.worksheet(f"{sym}").update('A1:O1', [['zone','status','open_id','open_date','open_price','side','size',
                                                                     'sl','tp','close_id','close_date','close_price','comment','profit']])
            print(f'----worksheet {sym} create')
 
        if (f"{self.symbol}_history" not in str(worksheet)):
            self.sheet.add_worksheet(title=f"{sym}_history", rows="100", cols="13")
            self.sheet.worksheet(f"{sym}_history").update('A1:O1', [['zone','status','open_id','open_date','open_price','side','size',
                                                                     'sl','tp','close_id','close_date','close_price','comment','profit']])
            print(f'----worksheet {sym}_history create')

        self.order = self.sheet.worksheet(f"{self.symbol}").get_all_records()
        self.summary = self.sheet.worksheet(f"summary").get_all_records()
        
    def create_log(self,res_order):
        #'symbol': 'THB_BTC', 'order_row': 2, 'history_row': 0
        self.summary = self.sheet.worksheet(f"summary").get_all_records()
        ord_row = self.summary['symbol']['order_row']+1
        his_row = self.summary['symbol']['history_row']+1

        if(status=='open'):
            self.sheet.worksheet(f"{self.summary['symbol']}").update(f'A{ord_row}:O{ord_row}',[res_order])
        
        elif(status=='close'):
            self.sheet.worksheet(f"{self.summary['symbol']}_history").update(f'A{his_row}:O{his_row}',[res_order])
            for i in range(len(self.order)):
                if(self.order[i]['zone'] == res_order['zone']
                  and self.order[i]['open_id']==res_order['open_id']):
                    self.sheet.worksheet.delete_row(i+1)
                    break

In [86]:
class main():
    def __init__(self,sym,amt,margin):
        #self.API = bitkubAPI(API_HOST,API_KEY,API_SECRET)
        self.API = ftxAPI()
        self.summary=[]
        self.order = {}
        self.margin = margin
        self.amt = amt
        self.his_price = {}
        self.MA = 10258
        self.ATR = 1
        
        self.symbol_1 = symbol('XRP-PERP',self.API)
        self.symbol_2 = symbol('XLM-PERP',self.API)
        self.his_price_1 = {}
        self.his_price_2 = {}
        #self.zone = np.arange((self.MA//margin)*margin,((self.MA//margin)*margin)+(margin*self.ATR*self.amt),margin) 
        
        gc = gspread.service_account(os.getcwd()+'\\service_account.json')
        self.sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/16sEZIsXLYYOrjVKo8xLOpusOeeYGAmnw_Xnrm0y2qwg/edit?usp=drive_web&ouid=112544876186489015703')
        self.system = False

        

            
    def start(self):
        #get_ticker
        sys_1 = self.symbol_1.get_ticker()
        sys_2 = self.symbol_2.get_ticker()
        self.system = all([sys_1,sys_2])
        #get_time
        tm = time.localtime() # get struct_time
        time_string = time.strftime("%Y/%m/%d, %H:%M:%S", tm)
        #get_hisbar
        self.previous_bar=12
        
        #-----start-----
        if(self.system):

            #------open condition ------
            #------check time
            time_check =all([ tm.tm_min//5==0 , tm.tm_sec == 0])
            #------check range
                #ask > ma
                #ask < previous_bar
            range_check = all([ self.ticker['ask'] >= self.MA, 
                                self.ticker['ask'] - self.previous_bar < 0])
            if(range_check and time_check):
                #------check zone
                range_zone = list(filter(lambda zone:zone <= self.previous_bar and  zone >= self.ticker['ask'], self.zone))

                #------check order   
                active_order=[]
                for order in self.order:
                    active_order.append(order["zone"])
                
                #-----cal action zone
                action_zone = list(set(range_zone) - set(active_order))
                
                if(action_zone > 0):
                    for zone in action_zone:
                        res=self.bitkub.place_order(self.symbol,'buy',self.amt,self.ticker['ask'] ,'market')
                        if(res['error'] == 0):
                            res_order = {
                                'zone':zone,
                                'status':'open',
                                'side':'buy',
                                'open_id' : res['result']['id'],
                                'open_date': res['result']['ts'],
                                'open_amt': res['result']['amt'],
                                'open_price': res['result']['rat'],
                                'open_rec': res['result']['rec'],
                                'sl': '',
                                'tp': self.ticker['ask'] + self.margin,
                                'fee':'',
                                'close_id':'',
                                'close_date':'',
                                'close_price':'',
                                'close_rec':'',
                                'comment':'',
                                'profit':''
                            }
                            self.order.append(res_order)
                            self.create_log(res_order)
                            
                                    
            #------close condition------
            for i in range(len(self.order)):
                close_order_condition = all([ self.order[i]['tp'] >= self.ticker['bid']])
                if(close_order_condition):
                    res=self.bitkub.place_order(self.symbol,'sell',self.order[i]['open_amt'],self.ticker['bid'] ,'market')
                    if(res['error'] == 0):
                        order = {
                            'zone':self.order[i]['zone'],
                            'status':'close',
                            'side':self.order[i]['side'],
                            'open_id' : self.order[i]['open_id'],
                            'open_date': self.order[i]['open_date'],
                            'open_amt': self.order[i]['open_amt'],
                            'open_price': self.order[i]['open_price'],
                            'open_rec': self.order[i]['open_rec'],
                            'sl': self.order[i]['sl'],
                            'tp': self.order[i]['tp'],
                            'fee':self.order[i]['fee'],
                            'close_id':res['id'],
                            'close_date':res['tm'],
                            'close_price':res['rat'],
                            'close_rec':res['rec'],
                            'comment':self.order[i]['comment'],
                            'profit':res['rec'] - self.order[i]['open_amt']
                            }
                        self.create_log(order)
                        self.order.pop(i)
                        break
                        
                        
            ask  = self.ticker['ask']
            print(f'{self.symbol} ASK:{ask} {time_string}    ',end='\r')
        print(f'connection failed {time_string}              ',end='\r')


In [87]:
THB_BTC=main('THB_BTC',100,1000)

In [88]:
while(True):
    clear_output(wait=True)
    THB_BTC.start()
    time.sleep(1)

TypeError: unsupported operand type(s) for -: 'float' and 'dict'

In [91]:
###################################################################

In [119]:
import time
import hmac
from typing import Optional, Dict, Any, List
from requests import Request, Session, Response

class ftxAPI():
    def __init__(self):
        self._api_host = 'https://ftx.com/api'
        self._api_key = b'1vyg6pC1WY-ODMhSyeHasYBV8ZVrgpAKYEu4uqWZ'
        self._api_secret  = 'qXuNtAcsQfZZEbzHlrm_rdBOtCrGgFHrYjdkshw8'
        self.ts = int(time.time() * 1000)
        self._subaccount_name = ""
        self._session = Session()
        
    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)
    
    def _post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('POST', path, params=params)
    
    def _request(self, method: str, path: str, **kwargs) -> Any:
        request = Request(method, self._api_host + path, **kwargs)
        self._sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _sign_request(self, request: Request) -> None:
        self.ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{self.ts}{prepared.method}{prepared.path_url}'.encode()
        if prepared.body:
            signature_payload += prepared.body
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(self.ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)
    
    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    
    def ticker(self,market_name):
        return self._get(f'/markets/{market_name}/orderbook?depth={1}')

    def place_orders(self,market,side,size,price,type_ord):
        return self._post(f'/orders',{   'market': market,
                                         'side': side,
                                         'price': price,
                                         'size': size,
                                         'type': type_ord,
                                     })
    
    def place_conditional_orders(self,market,side,size,type_ord,tp):
        return self._post(f'/conditional_orders',{    'market': market, 
                                                      'side': side,
                                                      'size': size,
                                                      'type': type_ord,
                                                      'triggerPrice':tp
                                                      'orderPrice':tp
                                                 })
    
    def historicalPrice(self,market_name,resolution,limit,start_time,end_time):
        return self._get(f'/markets/{market_name}/candles?resolution={resolution}&limit={limit}&start_time={start_time}&end_time={end_time}')
    
    def get_open_orders(self,market):
        return self._get(f'/orders?market={market}')
    
    def get_open_conditional_order(self,market):
        return self._get(f'/conditional_orders?market={market}')
    
    def get_conditional_order(self,conditional_order_id):
        return self._get(f'/conditional_orders/{conditional_order_id}/triggers')

In [120]:
ftx = ftxAPI()

In [149]:
ftx.ticker('XRP-PERP')

{'bids': [[0.472925, 800.0]], 'asks': [[0.47295, 677.0]]}

In [139]:

hisPrice=ftx.historicalPrice('XRP-PERP',300,100,int(time.time())-30000,int(time.time()))
hisPrice[len(hisPrice)-1]

{'startTime': '2021-03-18T10:05:00+00:00',
 'time': 1616061900000.0,
 'open': 0.4705,
 'high': 0.470675,
 'low': 0.4704,
 'close': 0.4705,
 'volume': 422.134425}

In [134]:
ts_now = 

In [135]:
ts_now

1616061817